# Imports
As we're luckily standing on the shoulders of giants, we can do some imports.

In [13]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from collections import defaultdict
import numpy as np
import imageio
import scipy.io as sio
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json

# Load Data
Let's load and convert the data, so we can use it later.

In [15]:
DATASETS = ['cifar10', 'mnist', 'cub', 'awa2',
            'imagenetfeatures', 'apyfeatures']
dataset = 'cub'

data_path = '/home/swezel/projects/urdtc/data/'

# attribute name lookup (first attr_id is 0) 
with open (data_path + 'cub/attributes.txt', 'r') as f:
    attributes=f.readlines()
attribute_name_dict = {str(int(attr.split(' ')[0])-1): attr.split(' ')[1] for attr in attributes}

def get_dataset_config(dataset, cnn_type, max_iters):
    input_channels = None
    if dataset == 'mnist':
        input_channels = 1
        if cnn_type == 'cnn':
            cnn_output_size = 4*4*100
        elif cnn_type == 'resnet':
            cnn_output_size = 512
        elif cnn_type == 'shallowcnn':
            cnn_output_size = 4*4*64
        out_freq = 100
        #assert max_iters > 4
    elif dataset == 'cifar10':
        input_channels = 3
        if cnn_type == 'cnn':
            cnn_output_size = 8*8*32
        elif cnn_type == 'resnet' or cnn_type == 'resnet18':
            cnn_output_size = 512
        elif cnn_type == 'shallowcnn':
            cnn_output_size = 4*4*64
        out_freq = 100
        #assert max_iters > 4
    elif dataset == 'cub':
        input_channels = 3
        if cnn_type == 'cnn':
            # cnn_output_size = 32*32*32
            cnn_output_size = 280900 # the above does not work? Maybe because of dataloader issue?
        elif cnn_type == 'resnet' or cnn_type == 'resnet152':
            cnn_output_size = 2048
        out_freq = 10
        #assert max_iters > 8
    elif dataset == 'awa2':
        input_channels = 3
        if cnn_type == 'cnn':
            cnn_output_size = 32*32*32  # TODO: check
        elif cnn_type == 'resnet' or cnn_type == 'resnet152':
            cnn_output_size = 2048
        out_freq = 10
        #assert max_iters > 6
    elif dataset == 'imagenetfeatures':
        cnn_output_size = 2048
        out_freq = 100
        #assert max_iters > 10
    elif dataset == 'apyfeatures':
        cnn_output_size = 2048
        out_freq = 10
        #assert max_iters > 5

    return input_channels, cnn_output_size, out_freq

class DataLoader(object):
    def __init__(self, dataset='mnist'):
        assert dataset in DATASETS
        self.dataset = dataset

    def load_data(self, batch_size=100, num_workers=4, root='./data/'):

        if self.dataset == 'mnist':
            #transform_train = transforms.ToTensor()
            #transform_test = transforms.ToTensor()
            class AddGaussianNoise(object):
                def __init__(self, mean=0., std=1.):
                    self.std = std
                    self.mean = mean

                def __call__(self, tensor):
                    output = tensor + torch.randn(tensor.size()) * self.std + self.mean
                    return output.clamp(0., 1.)

                def __repr__(self):
                    return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

            transform_train = transforms.Compose([
               transforms.ToTensor(),
               #AddGaussianNoise(0., 0.2)
               #transforms.Normalize((0.1307,), (0.3081,))
            ])
            transform_test = transforms.Compose([
               transforms.ToTensor(),
               #AddGaussianNoise(0., 0.2)
               #transforms.Normalize((0.1307,), (0.3081,))
            ])
            classes = [i for i in range(10)]
            dataset_class = dsets.MNIST

        elif self.dataset == 'cifar10':
            transform_train = transforms.Compose([
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])
            transform_test = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])
            classes = ('plane', 'car', 'bird', 'cat',
                       'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
            dataset_class = dsets.CIFAR10

        elif self.dataset == 'cub':

            transform_train = transforms.Compose([transforms.ToPILImage(),
                                                  transforms.RandomResizedCrop(224),
                                                  transforms.RandomHorizontalFlip(),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                                                       std=(0.229, 0.224, 0.225))])

            transform_test = transforms.Compose([transforms.ToPILImage(),
                                                 transforms.Resize(224),
                                                 transforms.CenterCrop(224),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                                                      std=(0.229, 0.224, 0.225))])

            dataset_class = CUB
            classes = list(range(200))

        elif self.dataset == 'awa2':
            transform_train = transforms.Compose([transforms.ToPILImage(),
                                                  transforms.RandomResizedCrop(224),
                                                  transforms.RandomHorizontalFlip(),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                                                       std=(0.229, 0.224, 0.225))])

            transform_test = transforms.Compose([transforms.ToPILImage(),
                                                 transforms.Resize(224),
                                                 transforms.CenterCrop(224),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                                                      std=(0.229, 0.224, 0.225))])

            dataset_class = AWA2
            classes = list(range(50))

        elif self.dataset == 'apyfeatures':
            transform_train = transforms.ToTensor()
            transform_test = transforms.ToTensor()

            dataset_class = APYFeatures
            classes = list(range(32))

        elif self.dataset == 'imagenetfeatures':
            transform_train = transforms.ToTensor()
            transform_test = transforms.ToTensor()

            dataset_class = ImageNetFeatures
            classes = list(range(1000))

        train_dataset = dataset_class(root=root,
                                      train=True,
                                      transform=transform_train,
                                      download=True)

        test_dataset = dataset_class(root=root,
                                     train=False,
                                     transform=transform_test)

        val_size = int(len(train_dataset) * 0.1)
        train_size = len(train_dataset) - val_size

        train_dataset, val_dataset = torch.utils.data.dataset.random_split(train_dataset, [train_size, val_size])

        train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   num_workers=num_workers)

        val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                                 batch_size=batch_size,
                                                 shuffle=False,
                                                 num_workers=num_workers)

        test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  num_workers=num_workers)

        dataloaders = {'train': train_loader,
                       'val': val_loader,
                       'test': test_loader}

        return dataloaders, classes

class CUB(Dataset):
    """CUB200-2011 dataset."""
    attribute_file = 'attributes/class_attribute_labels_continuous.txt'

    def __init__(self, root, train=True, transform=None, normalize=True,
                 download=None):
        self.root = os.path.join(root, 'cub')
        self.train = train
        self.transform = transform
        self.data_dir = os.path.join(self.root, 'images')

        train_test_split = pd.read_csv(os.path.join(self.root, 'train_test_split.txt'),
                                       sep=' ', index_col=0, header=None)
        if train:
            is_train_image = 1
        else:
            is_train_image = 0
        self.img_ids = train_test_split[train_test_split[1] == is_train_image].index.tolist()
        self.id_to_img = pd.read_csv(os.path.join(self.root, 'images.txt'),
                                     sep=' ', index_col=0, header=None)

        raw_mtx = np.loadtxt(os.path.join(self.root,
                                          self.attribute_file))
        raw_mtx[raw_mtx == -1] = 0
        raw_mtx = raw_mtx / raw_mtx.max()
        self.attribute_mtx = torch.tensor(raw_mtx, dtype=torch.float)

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_name = self.id_to_img[self.id_to_img.index == img_id].values[0][0]
        img_path = os.path.join(self.data_dir, img_name)

        img = imageio.imread(img_path, pilmode='RGB')
        label = int(img_name[:3]) - 1

        if self.transform:
            img = self.transform(img)

        return img, label, img_path

In [16]:
device = torch.device('cuda')
# device = torch.device('cpu')

# create dataloader objects for train, val and test
dl = DataLoader(dataset=dataset)
# dataloaders, classes = dl.load_data(4, 4, data_path)# 128 insted of 
dataloaders, classes = dl.load_data(64, 4, data_path)# 128 insted of 

# attributes (312 column vectors with 200 rows) -> each class can be described with 312 attributes
# percentage of time, human annotator thought, the attribute was present
attribute_mtx = dataloaders['train'].dataset.dataset.attribute_mtx

# create binary encoding for class attributes
attribute_mtx[attribute_mtx < 0.5] = 0.0
attribute_mtx[attribute_mtx >= 0.5] = 1.0
attribute_mtx = attribute_mtx.to(device) # cuda
attribute_size = attribute_mtx.size(1) # number of available attributes

FileNotFoundError: [Errno 2] File /home/swezel/projects/urdtc/data/cub/train_test_split.txt does not exist: '/home/swezel/projects/urdtc/data/cub/train_test_split.txt'